In the previous notebook we tried to model using the Serf Exclusion variable. We also foud a subset of 4 variables that provided a high accuracy for predicting the Serf Exclusion Flag. Given the success of this approach, can we now model the Risk Rating?

The purpose of this notebook is to use classifiers to classify the risk rating

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [19]:
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
df = pd.read_csv('df_client.csv')

In [4]:
df.head()

,Country_Code,BR Code,Period,Client,risk_rating,Self_exclude_flag,Variable_1_Y0,Variable_1_Y1,Variable_1_Y2,Variable_1_Y3,...,Variable_28_Y1,Variable_28_Y2,Variable_28_Y3,Variable_29_Y0,Variable_29_Y1,Variable_29_Y2,Variable_30_Y0,Variable_30_Y1,Variable_30_Y2,Variable_30_Y3
0,0,0,2017Q2,0,7,1,581103.4591,612122.5165,589483.6484,608043.5063,...,572312.4225,601762.9316,574251.4130,577170.3096,594024.8975,616177.8226,588163.8327,623659.1015,608794.9055,574860.551
1,0,0,2016Q1,0,7,1,608189.3682,581513.6158,609292.1500,NaN,...,608263.6088,605605.1646,NaN,581951.0166,608354.2362,623470.1198,591055.8212,592011.4052,572734.0028,NaN
2,0,0,2015Q4,0,7,1,626775.4450,620338.8464,NaN,NaN,...,621396.2940,NaN,NaN,590490.3620,620329.2616,NaN,626221.0887,572241.0321,NaN,NaN
3,0,0,2015Q2,0,7,1,613152.4469,595630.8819,NaN,NaN,...,589714.2432,NaN,NaN,580633.8747,576235.2813,NaN,619098.6619,578761.7137,NaN,NaN
4,0,1,2019Q1,1,9,0,615840.2415,603501.2067,587601.9393,610071.5454,...,607400.3547,570273.9177,573434.8221,572413.5987,618435.4264,587802.7283,NaN,NaN,NaN,NaN


In [22]:
#cols = ['Variable_16_Y0','Variable_17_Y0', 'Variable_22_Y0','Variable_3_Y0', 
cols = ['Variable_16_Y0','Variable_3_Y0', 
        'risk_rating']

In [79]:
df1 = df.loc[:,cols].dropna().sort_values(by='risk_rating').reset_index(drop=True)

In [81]:
df1.head()

,Variable_16_Y0,Variable_3_Y0,risk_rating
0,9.659202e+08,597439.6848,1
1,6.167195e+05,626093.3911,1
2,5.983304e+05,592918.6650,1
3,9.681877e+08,626715.8246,1
4,1.043984e+09,613926.4115,1


In [83]:
pd.value_counts(df1['risk_rating'])

7     4816
6     4604
8     4216
5     3312
4     2143
9     1660
10     718
3      652
11     437
12     199
2      178
13     139
17     109
14      27
15      16
1        9
Name: risk_rating, dtype: int64

In [84]:
remove_rating = [1,14,15,17]

In [85]:
# split into inputs and outputs
f1 = ~df1['risk_rating'].isin(remove_rating)

X, y = df1.loc[f1,cols[:-1]], df1.loc[f1,cols[-1]]
print(X.shape, y.shape)

(23074, 2) (23074,)


In [91]:
pd.value_counts(y)

7     4816
6     4604
8     4216
5     3312
4     2143
9     1660
10     718
3      652
11     437
12     199
2      178
13     139
Name: risk_rating, dtype: int64

In [103]:
np.random.choice(y[y==2].index.values,10,replace=False)

array([133,  23,  82,  49, 130,  72,  98,  71, 128,  12], dtype=int64)

In [114]:
def test_train_split(y):
    vc = pd.value_counts(y)
    ind = vc.index.values

    l = 0
    cnt = 0
    
    for item in ind:
        
        ser = y[y==item]
        l = len(ser)
        
        if cnt == 0:
            train_ix = np.random.choice(ser.index.values,round(0.8*l),replace=False)
            cnt = cnt+1
        else:
            train_ix = np.append(train_ix,
                                 np.random.choice(ser.index.values,round(0.8*l),replace=False))
        
    return train_ix

We cannot use the standard Scikit Learn module to split the dataset into Test and Train datasets. This is because we want to ensure that all the classes are represented in the Test and Train datasets. The code above was written to accomplish this.

In [115]:
train_ix = test_train_split(y)

In [117]:
train_ix.shape

(18459,)

In [118]:
X_train = X.loc[X.index.isin(train_ix),:]
y_train = y.loc[y.index.isin(train_ix)]

X_test = X.loc[~X.index.isin(train_ix),:]
y_test = y.loc[~y.index.isin(train_ix)]

In [119]:
pd.value_counts(y_train)

7     3853
6     3683
8     3373
5     2650
4     1714
9     1328
10     574
3      522
11     350
12     159
2      142
13     111
Name: risk_rating, dtype: int64

In [120]:
pd.value_counts(y_test)

7     963
6     921
8     843
5     662
4     429
9     332
10    144
3     130
11     87
12     40
2      36
13     28
Name: risk_rating, dtype: int64

In [121]:
# fit the model
model = RandomForestClassifier(random_state=1)
model.fit(X_train, y_train.values)

RandomForestClassifier(random_state=1)

In [122]:
# make predictions
yhat = model.predict(X_test)

In [123]:
# evaluate predictions
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.168


The accuracy of the model is very low

In [124]:
cm = confusion_matrix(y_test, yhat) 
print ("Confusion Matrix : \n", cm) 

Confusion Matrix : 
 [[  1   1   2   6   7  11   5   1   2   0   0   0]
 [  1   1   6  21  29  31  33   6   2   0   0   0]
 [  1  10  25  82 106  90  80  23   6   5   1   0]
 [  4   8  47 120 143 159 134  32   8   3   4   0]
 [  6   9  62 126 205 263 187  35  10  13   3   2]
 [  4  15  74 138 211 238 190  58  18  13   2   2]
 [  5  12  61 121 212 212 155  43  13   6   2   1]
 [  1   2  28  34  72  98  61  22   4   6   2   2]
 [  1   3   3  25  40  32  23   7   7   2   1   0]
 [  0   1   5   8  24  24  17   5   2   0   0   1]
 [  0   0   1   7  12   9   8   3   0   0   0   0]
 [  0   0   1   3   8  13   2   1   0   0   0   0]]


In [125]:
unique, counts = np.unique(yhat, return_counts=True)

print("yhat : \n", np.asarray((unique, counts)).T)

yhat : 
 [[   2   24]
 [   3   62]
 [   4  315]
 [   5  691]
 [   6 1069]
 [   7 1180]
 [   8  895]
 [   9  236]
 [  10   72]
 [  11   48]
 [  12   15]
 [  13    8]]


In [126]:
unique, counts = np.unique(y_test, return_counts=True)

print("y_test : \n", np.asarray((unique, counts)).T)

y_test : 
 [[  2  36]
 [  3 130]
 [  4 429]
 [  5 662]
 [  6 921]
 [  7 963]
 [  8 843]
 [  9 332]
 [ 10 144]
 [ 11  87]
 [ 12  40]
 [ 13  28]]
